<a href="https://colab.research.google.com/github/santoshcbit2002/ApacheBeamDemo/blob/master/Side_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install apache_beam

     |████████████████████████████████| 8.3MB 2.7MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 1.4MB 39.4MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 81kB 5.8MB/s 
     |████████████████████████████████| 63.8MB 101kB/s 
     |████████████████████████████████| 153kB 47.7MB/s 
     |████████████████████████████████| 829kB 38.7MB/s 
     |████████████████████████████████| 112kB 41.9MB/s 
  Created wheel for avro-python3: filename=avro_python3-1.9.2.1-cp36-none-any.whl size=43516 sha256=5e29d4d78750637c3ad527e1ffd70c98c1a082621dabe755f48bc0d9914c5ece
  Stored in directory: /root/.cache/pip/wheels/5a/d3/be/86620c9dd3fca68986c33b9c616510289fc0abb81ec9aa70bd
  Created wheel for hdfs: filename=hdfs-2.5.8-cp36-none-any.whl size=33213 sha256=e6459022c1241983f6769682a3307575fc8fe5ba0aefb2283c64d898005fc40f
  Stored in directory: /root/.cache/pip/wheels/fe/a7/05/23e3699975fc20f8a30e00ac1e51

In [2]:
!mkdir data
!ls

data  sample_data


In [3]:
from google.colab import files
upload=files.upload()

Saving exclude_ids.txt to exclude_ids.txt


In [4]:
from google.colab import files
upload=files.upload()

Saving dept_data.txt to dept_data.txt


In [5]:
!ls

data  dept_data.txt  exclude_ids.txt  sample_data


In [8]:
import apache_beam as beam

side_list=list()
with open ('exclude_ids.txt','r') as my_file:
  for line in my_file:
    side_list.append(line.rstrip())

p = beam.Pipeline()

# We can pass side inputs to a ParDo transform, which will 
# get passed to its process method.
# The first two arguments for the process method would be self and element.
# The side input has a list of employee ids that should be selected and also a 
# boundary on the names that will be applied. In this case, the names of 
# employee should be between 3 and 10.

class FilterUsingLength(beam.DoFn):
  def process(self, element,side_list,lower_bound, upper_bound=float('inf')):
    id = element.split(',')[0]
    name = element.split(',')[1]
    element_list= element.split(',')
    if (lower_bound <= len(name) <= upper_bound) and id not in side_list:
      return [element_list]

# using pardo to filter names with length between 3 and 10
small_names =( 
            p
          | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')
          | "ParDo with side inputs" >> beam.ParDo(FilterUsingLength(),
                                                   side_list,3,10) 
          | beam.Filter(lambda record: record[3] == 'Accounts')
          | beam.Map(lambda record: (record[0]+ " " + record[1], 1))
          | beam.CombinePerKey(sum)
          | 'Write results' >> beam.io.WriteToText('data/output_new_final')
             )

p.run()

!{('head -n 20 data/output_new_final-00000-of-00001')}

('503996WI Edouard', 31)
('957149WC Kyle', 31)
('241316NX Kumiko', 31)
('796656IE Gaston', 31)
('718737IX Ayumi', 30)
